# Accumulation and Ablation

Goals of this notebook:

- gain a basic understanding of accumulation, ablation and glacier mass balance
- understand the link between mass balance and ice flow
- implement a simple experiment to calculate ice flow on a glacier in equilibrium

In [ ]:
import oggm
from oggm import cfg
import oggm_edu as edu
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import numpy as np
from IPython.display import Image
from functools import partial

In [ ]:
cfg.initialize()

In this notebook, we will essentially reconstruct the series of images [here](http://edu.oggm.org/en/latest/glacier_basics.html#) using OGGM.

## Set the scene

In the introduction on the OGGM-Edu [website](http://edu.oggm.org/en/latest/glacier_basics.html), a cross section of a typical mountain glacier is shown.

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_01.png', width=600)

Such a glacier can be reconstructed in OGGM, as done in the [flowline model notebook](flowline_model.ipynb). First, we define a linear bedrock profile:

In [ ]:
# define horizontal resolution of the model:
# nx: number of grid points
# map_dx: grid point spacing in meters
nx = 200
map_dx = 100

In [ ]:
# define glacier top and bottom altitudes in meters
top = 3400
bottom = 1400

In [ ]:
# create a linear bedrock profile from top to bottom
bed_h, surface_h = edu.define_linear_bed(top, bottom, nx)

In [ ]:
# calculate the distance from the top to the bottom of the glacier in km
distance_along_glacier = edu.distance_along_glacier(nx, map_dx)

To initialize the glacier, we need to specify how wide the glacier is. Here, we will use a wider accumulation area and a narrow ablation area, typical for mountain glaciers. The accumulation area is determined by the following thresholds of its width at the top of the glacier and at the ELA:

In [ ]:
ACCW = 400
ELAW = 200
AAR = 5

In [ ]:
# accumulation area occupies a fraction of 1 / AAR the glacier extent
acc_width = np.linspace(ACCW, ELAW, int(nx / AAR))

In [ ]:
abl_width = np.tile(ELAW, nx-len(acc_width))

In [ ]:
widths = np.hstack([acc_width, abl_width])

The upper part of the glacier linearly decreases in width and the lower part of the glacier has constant width. For the model run, the widths have to be converted to grid point space.

In [ ]:
mwidths = np.zeros(nx) + widths / map_dx

Now, we need to define a glacier bedrock shape - here, we use a rectangular bed:

In [ ]:
# define our bed
init_flowline = oggm.core.flowline.RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h, widths=mwidths, map_dx=map_dx)

Then we will need a mass balance model. In our case this will be a simple linear mass balance, defined by the equilibrium line altitude (ELA) and a linear mass balance gradient with respect to elevation (in [mm m$^{-1}$]). The equilibrium line altitude is located at the transition between the accumulation and ablation zone, as discussed above.

In [ ]:
# equilibrium line altitude
ela = bed_h[np.where(widths==ELAW)[0][0]]
print('ELA: {:.2f} m'.format(ela))

In [ ]:
# mass balance gradient with respect to elevation in mm/m
altgrad = 5
mb_model = oggm.core.massbalance.LinearMassBalance(ela, grad=altgrad)

Now that we have all the ingredients to run the model, we just have to initialize it:

In [ ]:
# The model requires the initial glacier bed, a mass balance model, and an initial time (the year y0)
model = oggm.core.flowline.FluxBasedModel(init_flowline, mb_model=mb_model, y0=0., min_dt=0, cfl_number=0.01)

Let's run the model until our test glacier is in equilibrium.

In [ ]:
model.run_until_equilibrium(max_ite=500)
edu.glacier_plot(x=distance_along_glacier, bed=bed_h, model=model, mb_model=mb_model, init_flowline=init_flowline)

The glacier surface along the defined flowline is:

In [ ]:
initial = model.fls[-1].surface_h

Now, we have set the scene to explore the ice flow of a glacier in equilibrium.

In [ ]:
# X, Y = np.meshgrid(distance_along_glacier, widths)
# Z = np.tile(gsurface, (nx, 1))
# fig = plt.figure(figsize=(16, 9))
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(X, Y, Z)
# ax.plot_surface(X, Y, np.tile(bed_h, (nx, 1)))

## Accumulation and Ablation

For a glacier to be in equilibrium, we require the mass balance (accumulation + ablation) to be zero over an integration period. To check this requirement, we can use the mass balance model to compute the annual mass balance and compute a width weighted average over all altitudes:

In [ ]:
gmb = mb_model.get_annual_mb(initial) * cfg.SEC_IN_YEAR
print('Total glacier mass balance: {:.2f} m'.format(np.average(gmb, weights=widths)))

In the previous section, we defined a constant mass balance gradient

$$\frac{\partial \dot{m}}{\partial z} = c,$$

which gives

$$\dot{m}(z) = \dot{m}(z_0) + c (z - z_0).$$

For simplicity, we choose the reference height $z_0$ to be the height of the equilibrium line altitude and hence $\dot{m}(z_0) = 0$.

Assume, that this glacier now gains mass over the accumulation season - the mass gain increases linearly with height from the terminus to the top as shown in the image below:

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_03.png', width=600)

Here, we use unrealistically large accumulations to improve visual perception:

In [ ]:
# accumulation at the glacier terminus: m / yr
acc_0 = 100

In [ ]:
# glacier terminus
terminus = initial[initial == bed_h][0]

In [ ]:
# accumulation as a function of altitude
acc = acc_0 + altgrad * (bed_h[bed_h > terminus] - terminus) * 1e-3

In [ ]:
# append 0 accumulation downstream of glacier terminus
acc = np.hstack([acc, np.zeros(len(bed_h[bed_h <= terminus]))])

Applying this accumulation to our initial glacier surface in equilibrium:

In [ ]:
acc_sfc = initial + acc

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(distance_along_glacier, bed_h, '--k', label='bed')
ax.plot(distance_along_glacier, initial, '--r', label='initial')
ax.plot(distance_along_glacier, acc_sfc, '-', label='accumulation')
ax.hlines(ela, distance_along_glacier[0], distance_along_glacier[-1], linestyle='--', color='grey')
ax.text(distance_along_glacier[-1], ela + 10, 'Equilibrium line altitude', horizontalalignment='right', verticalalignment='bottom', color='grey')
ax.fill_between(distance_along_glacier, bed_h, initial, color='grey', alpha=0.3);
ax.fill_between(distance_along_glacier, initial, acc_sfc, color='lightblue', alpha=0.5);
ax.legend(frameon=False);

Next, during the ablation season, the glacier looses mass according to the same linear mass balance gradient.

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_04.png', width=600)

In [ ]:
# ablation at terminus = accumulation at top: m / yr
abl_0 = - acc[0]

In [ ]:
# ablation as a function of altitude
abl = abl_0 + altgrad * (bed_h[bed_h >= terminus] - terminus) * 1e-3

In [ ]:
# append 0 accumulation downstream of glacier terminus
abl = np.hstack([abl, np.zeros(len(bed_h[bed_h < terminus]))])

Applying this accumulation to our initial glacier surface in equilibrium:

In [ ]:
# glacier surface after ablation
abl_sfc = initial + abl

In [ ]:
# correct where ice thickness is less than bedrock
no_ice = np.where(abl_sfc < bed_h)
abl_sfc[no_ice] = bed_h[no_ice]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(distance_along_glacier, bed_h, '--k', label='bed')
ax.plot(distance_along_glacier, initial, '--r', label='initial')
ax.plot(distance_along_glacier, acc_sfc, '-', label='accumulation')
ax.plot(distance_along_glacier, abl_sfc, '-r', linewidth=2, label='ablation')
ax.hlines(ela, distance_along_glacier[0], distance_along_glacier[-1], linestyle='--', color='grey')
ax.text(distance_along_glacier[-1], ela + 10, 'Equilibrium line altitude', horizontalalignment='right', verticalalignment='bottom', color='grey')
ax.fill_between(distance_along_glacier, bed_h, abl_sfc, color='grey', alpha=0.3);
ax.fill_between(distance_along_glacier, initial, acc_sfc, color='lightblue', alpha=0.5);
ax.fill_between(distance_along_glacier, initial, abl_sfc, color='red', alpha=0.3);
ax.legend(frameon=False);

## Mass Balance

The net mass balance is then just the sum of accumulation and ablation:

In [ ]:
mb_sfc = initial + acc + abl

In [ ]:
# correct where ice thickness is less than bedrock
no_ice = np.where(mb_sfc < bed_h)
mb_sfc[no_ice] = bed_h[no_ice]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(distance_along_glacier, bed_h, '--k', label='bed')
ax.plot(distance_along_glacier, initial, '--r', label='initial')
ax.plot(distance_along_glacier, mb_sfc, '-k', linewidth=2, label='mass balance')
ax.hlines(ela, distance_along_glacier[0], distance_along_glacier[-1], linestyle='--', color='grey')
ax.text(distance_along_glacier[-1], ela + 10, 'Equilibrium line altitude', horizontalalignment='right', verticalalignment='bottom', color='grey')
ax.fill_between(distance_along_glacier, bed_h, initial, initial <= mb_sfc, color='grey', alpha=0.3);
ax.fill_between(distance_along_glacier, bed_h, mb_sfc, initial >= mb_sfc, color='grey', alpha=0.3);
ax.fill_between(distance_along_glacier, initial, mb_sfc, mb_sfc >= initial, color='lightblue', alpha=0.5);
ax.fill_between(distance_along_glacier, initial, mb_sfc, mb_sfc < initial, color='red', alpha=0.3);
ax.legend(frameon=False);

In [ ]:
# if this is 0, glacier is in equilibrium
print('Net mass balance: {} m yr^-1'.format(np.average(mb_sfc - initial, weights=widths)))

## Ice flux

Generally, ice flux is determined by the continuity equation,

$$\frac{\partial S}{\partial t} = \dot{m} - \nabla \cdot \vec{q},$$

where, $S$ is the glacier surface, $\dot{m}$ the mass balance and $\vec{q}$ the ice flux. In equilibrium, glacier surface $S$ does not change and the continuity equation results to

$$\dot{m} = \nabla \vec{q}.$$

This means that glacier mass balance solely determines ice flux if the glacier is in steady-state. Hence, the ice flux can be computed by vertically integrating the mass balance - in the one-dimensional case, the ice flux is the sum of the mass balance above a certain reference height $z$.

In [ ]:
# width weighted mass balance removing the initial steady-state surface: m^2 / yr
mb = (mb_sfc - initial) * widths

In [ ]:
# ice flux through the equilibrium line: m^3 s^-1
mb[mb > 0].sum() / cfg.SEC_IN_YEAR